In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,Add, AveragePooling2D, Flatten, Dense,ZeroPadding2D,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/MRNet-v1.0" 

Archive:  /content/drive/My Drive/MRNet-v1.0.zip
warning [/content/drive/My Drive/MRNet-v1.0.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
   creating: MRNet-v1.0/
  inflating: MRNet-v1.0/valid-abnormal.csv  
   creating: MRNet-v1.0/valid/
   creating: MRNet-v1.0/valid/axial/
  inflating: MRNet-v1.0/valid/axial/1139.npy  
  inflating: MRNet-v1.0/valid/axial/1138.npy  
  inflating: MRNet-v1.0/valid/axial/1249.npy  
  inflating: MRNet-v1.0/valid/axial/1248.npy  
  inflating: MRNet-v1.0/valid/axial/1207.npy  
  inflating: MRNet-v1.0/valid/axial/1213.npy  
  inflating: MRNet-v1.0/valid/axial/1171.npy  
  inflating: MRNet-v1.0/valid/axial/1165.npy  
  inflating: MRNet-v1.0/valid/axial/1159.npy  
  inflating: MRNet-v1.0/valid/axial/1158.npy  
  inflating: MRNet-v1.0/valid/axial/1164.npy  
  inflating: MRNet-v1.0/valid/axial/1170.npy  
  inflating: MRNet

In [ ]:
def read_data(data_type, scan_type):
  list = []
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      mid = len(instance) // 2
      instance = np.array([ instance[mid-1] , instance[mid] ,instance[mid+1]])
      list.append(instance)
  data = np.asarray(list)
  data = np.moveaxis(data, 1, 3)
  print(scan_type+data_type, data.shape)
  return data

def read_labels(data_type, scan_type):
  path = "/content/MRNet-v1.0/"+data_type+"-"+scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  print(scan_type+data_type, labels.shape)
  return labels

print("Data: ")
sagittal_train_x = read_data("train","sagittal")
axial_train_x = read_data("train","axial")
coronal_train_x = read_data("train","coronal")

sagittal_test_x = read_data("valid","sagittal")
axial_test_x = read_data("valid","axial")
coronal_test_x = read_data("valid","coronal")

print("Labels: ")
abnormal_train_y = read_labels('train','abnormal')
acl_train_y = read_labels('train','acl')
meniscus_train_y = read_labels('train','meniscus')

abnornmal_test_y = read_labels('valid','abnormal')
acl_test_y = read_labels('valid','acl')
meniscus_test_y = read_labels('valid','meniscus')

Data: 
sagittaltrain (1130, 256, 256, 3)
axialtrain (1130, 256, 256, 3)
coronaltrain (1130, 256, 256, 3)
sagittalvalid (120, 256, 256, 3)
axialvalid (120, 256, 256, 3)
coronalvalid (120, 256, 256, 3)
Labels: 
abnormaltrain (1130,)
acltrain (1130,)
meniscustrain (1130,)
abnormalvalid (120,)
aclvalid (120,)
meniscusvalid (120,)


In [ ]:
acl_train_y=acl_train_y.reshape(-1,1)

In [ ]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(256, 256, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t=Dropout(0.5)(t)
    t = Flatten()(t)
    outputs = Dense(1, activation='sigmoid')(t)
    
    model = Model(inputs, outputs)

 

    return model

In [ ]:
def ResNet50(input_shape = (256, 256, 3), classes = 1):
    
    X_input = Input(input_shape)

    

    X = ZeroPadding2D((3, 3))(X_input)
    

    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)


    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')



    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')


    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D()(X)

    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    

    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
def identity_block(X, f, filters, stage, block):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters
    

    X_shortcut = X



    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer=glorot_uniform(seed =0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu') (X)

    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer=glorot_uniform(seed =0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer=glorot_uniform(seed =0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
model = ResNet50()
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer= sgd, metrics=['accuracy'])
checkpoint = ModelCheckpoint('drive/My Drive/sagital_acl.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model.fit(
    x=sagittal_train_x,
    y=abnormal_train_y,
    epochs=20,
    verbose=1,
    validation_split=0.1,
    batch_size=32,
    callbacks=[checkpoint]
)

Epoch 1/20
32/32 [==============================] - ETA: 0s - loss: 7.5178 - accuracy: 0.7168
Epoch 00001: val_accuracy improved from -inf to 0.80531, saving model to drive/My Drive/sagital_acl.h5
32/32 [==============================] - 11s 344ms/step - loss: 7.5178 - accuracy: 0.7168 - val_loss: 8848.5264 - val_accuracy: 0.8053
Epoch 2/20
32/32 [==============================] - ETA: 0s - loss: 1.8969 - accuracy: 0.7837
Epoch 00002: val_accuracy did not improve from 0.80531
32/32 [==============================] - 7s 207ms/step - loss: 1.8969 - accuracy: 0.7837 - val_loss: 843.0284 - val_accuracy: 0.8053
Epoch 3/20
32/32 [==============================] - ETA: 0s - loss: 0.8365 - accuracy: 0.8201
Epoch 00003: val_accuracy did not improve from 0.80531
32/32 [==============================] - 7s 207ms/step - loss: 0.8365 - accuracy: 0.8201 - val_loss: 635.7366 - val_accuracy: 0.7699
Epoch 4/20
32/32 [==============================] - ETA: 0s - loss: 0.6578 - accuracy: 0.8161
Epoch 0000

In [ ]:

#print("Loaded model from disk")

score = model.evaluate(sagittal_test_x, abnornmal_test_y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 84.17%


0.0

1
